In [ ]:
!pip install transformers[sentencepiece]
! pip install open_clip_torch matplotlib
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 46.3 MB/s 
     |████████████████████████████████| 6.6 MB 41.3 MB/s 
     |████████████████████████████████| 163 kB 66.0 MB/s 
     |████████████████████████████████| 1.3 MB 50.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 9.5 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")
model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefine

(768,)


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/university/test.csv')
df_test.head()

,id,description
0,486,Фотография. Елизавета Алексеевна Юманова. ПКМ...
1,813,Фотография. Заседание комитета комсомола мотор...
2,2980,"Фотография. День ""Саланга"". ПОКМ-18530/638 фо..."
3,13,Фотография. Елизавета Алексеевна Юманова. ПКМ...
4,2467,"Фотография. ""Универмаг Пермь. Оформление демон..."


In [ ]:
test_embeddings = []

for i, desc in tqdm(enumerate(df_test.description.values)):
    test_embeddings.append(embed_bert_cls(desc, model, tokenizer))
    # print(desc)
    # break

900it [00:18, 49.53it/s]


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/university/train.csv')
df_train.head()

,id,description,object_img
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244


In [ ]:
train_embeddings = []
for i, desc in tqdm(enumerate(df_train.description.values)):
    train_embeddings.append(embed_bert_cls(desc, model, tokenizer))

2098it [00:28, 73.48it/s]


In [ ]:
train_embeddings = list(map(lambda x: x[None, ...], train_embeddings))
test_embeddings = list(map(lambda x: x[None, ...], test_embeddings))


In [ ]:
train_concated = np.concatenate(train_embeddings, axis=0)
train_concated /= np.linalg.norm(train_concated, axis=1)[..., None]
test_concated = np.concatenate(test_embeddings)
test_concated /= np.linalg.norm(test_concated, axis=1)[..., None]

In [ ]:
# # from sklearn.neural_network import MLPRegressor
# # regr = MLPRegressor(random_state=1, max_iter=500).fit(train_concated, df_train.object_img.values)
# from catboost import CatBoostRegressor
# regr = CatBoostRegressor()
# # Fit model
# regr.fit(train_concated, df_train.object_img.values)

In [ ]:
mul = test_concated @ train_concated.T

In [ ]:
mul.shape

(900, 2098)

In [ ]:
def find_nearest(s, v):
    min_dif, min_val = 10000000, 100000000
    for i in s:
        if abs(i - v) < min_dif:
            min_dif = abs(i - v)
            min_val = i
    return min_val

def find_nearest_n(s, v):
    lst = list(s)
    lst.sort(key=lambda x: abs(x - v))
    return lst[:10]

In [ ]:
def parse_df_vecs(vec):
    return np.array(list(map(float, vec[1:-1].split(","))), dtype=float)[None, ...]

df_txt = pd.read_csv("/content/drive/MyDrive/university/df_txt.csv")
df_img = pd.read_csv("/content/drive/MyDrive/university/img_df.csv")

img_ids = df_img["id"].values
img_vecs = df_img["vec"].values

txt_ids = df_txt["id"].values
txt_vecs = df_txt["vec"].values 

img_vecs_2 = list(map(parse_df_vecs, img_vecs))
img_vecs_2 = np.concatenate(img_vecs_2, axis=0)
# img_vecs_2.shape

txt_vecs_2 = list(map(parse_df_vecs, txt_vecs))
txt_vecs_2 = np.concatenate(txt_vecs_2, axis=0)
txt_vecs_2.shape

image_featurs = img_vecs_2 / np.linalg.norm(img_vecs_2, axis=1)[..., None]
text_featurs = txt_vecs_2 / np.linalg.norm(txt_vecs_2, axis=1)[..., None]

test_img_id_2_vec = dict(tuple(zip(img_ids, image_featurs)))
test_txt_id_2_vec = dict(tuple(zip(txt_ids, text_featurs)))
# test_mul = image_featurs @ text_features

In [ ]:
from sklearn.neighbors import KDTree
kdt = KDTree(train_concated, leaf_size=30, metric='euclidean')

### find nearest n bert

In [ ]:
test_ids = set(list(map(lambda x: int(x.split('.')[0]), os.listdir('/content/drive/MyDrive/university/test'))))
train_ids = df_train.object_img.values
ans = []
for i in range(len(mul)):
    cur_line = mul[i]
    amax = np.argmax(cur_line)
    # nearest = find_nearest(test_ids, train_ids[amax])
    
    nearest_n = find_nearest_n(test_ids, train_ids[amax])

    nearest, max_dist = 1e9, -1
    for j in range(len(nearest_n)):
        dist = test_img_id_2_vec[nearest_n[j]] @ test_txt_id_2_vec[txt_ids[i]]
        if dist > max_dist:
            max_dist = dist
            nearest = nearest_n[j]
    # print()
    test_ids.remove(nearest)
    ans.append(nearest)

    # # print(amax)
    # break

### find nearest n regr

In [ ]:
test_ids = set(list(map(lambda x: int(x.split('.')[0]), os.listdir('/content/drive/MyDrive/university/test'))))
train_ids = df_train.object_img.values
ans = []
test_ids_df = df_test.id.values
nearest_dist_tuple = []

for i in range(len(mul)):
    cur_line = mul[i]
    amax = np.argmax(cur_line)
    nearest = find_nearest(test_ids, train_ids[amax])
    nearest_dist_tuple.append((abs(nearest - train_ids[amax]), i))

    # nearest_n = find_nearest_n(test_ids, regr.predict(test_concated[i][None, ...])[0])
    # print(regr.predict(test_concated[i][None, ...]))
    # break

    # nearest, max_dist = 1e9, -1
    # for j in range(len(nearest_n)):
    #     dist = test_img_id_2_vec[nearest_n[j]] @ test_txt_id_2_vec[txt_ids[i]]
    #     if dist > max_dist:
    #         max_dist = dist
    #         nearest = nearest_n[j]
    # # print()
    # test_ids.remove(nearest)
    # ans.append(nearest)

    # # print(amax)
    # break
nearest_dist_tuple.sort()
ans_ids_text = []
for i in range(len(mul)):
    cur_line = mul[nearest_dist_tuple[i][1]]
    amax = np.argmax(cur_line)
    # nearest = find_nearest(test_ids, train_ids[amax])
    # nearest_tuple.append((abs(nearest - train_ids[amax]), i))

    nearest_n = find_nearest_n(test_ids, train_ids[amax])

    nearest, max_dist = 1e9, -1
    for j in range(len(nearest_n)):
        dist = test_img_id_2_vec[nearest_n[j]] @ test_txt_id_2_vec[txt_ids[i]]
        if dist > max_dist:
            max_dist = dist
            nearest = nearest_n[j]
    # print()
    test_ids.remove(nearest)
    ans.append(nearest)
    ans_ids_text.append(test_ids_df[nearest_dist_tuple[i][1]])

# for i in range(len())

In [ ]:
# test_ids = set(list(map(lambda x: int(x.split('.')[0]), os.listdir('/content/drive/MyDrive/university/test'))))
# train_ids = df_train.object_img.values
# ans = []
# for i in range(len(mul)):
#     cur_line = mul[i]
#     # amax = np.argmax(cur_line)
#     cur_txt_emb = test_concated[i]
#     q = kdt.query([cur_txt_emb], k=5, return_distance=False)
#     train_id_find = []
#     for i in range(len(q[0])):
#         train_id_find.append(train_ids[q[0][i]])
#     # print(train_id_find)
#     # break
#     # nearest = find_nearest(test_ids, train_ids[amax])
#     nearest_n = find_nearest_n(test_ids, np.mean(np.array(train_id_find)))

#     nearest, max_dist = 1e9, -1
#     for j in range(len(nearest_n)):
#         dist = test_img_id_2_vec[nearest_n[j]] @ test_txt_id_2_vec[txt_ids[i]]
#         if dist > max_dist:
#             max_dist = dist
#             nearest = nearest_n[j]
#     # print()
#     test_ids.remove(nearest)
#     ans.append(nearest)

#     # # print(amax)
#     # break

In [ ]:
set(ans_ids_text) == set(test_ids_df)

True

In [ ]:
fin = pd.DataFrame()
fin["id"] = ans_ids_text
fin["object_img"] = ans
fin.to_csv("ans_top_10_with_sort_fix_ind.csv", index=False)